In [15]:
!pip install  -q pyiceberg
!pip install  -q deltalake
%pip install  -q getdaft

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 35, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [16]:
from   pyiceberg.catalog import load_catalog
import os
from   datetime import datetime
import glob
import re 
import requests
from   shutil import unpack_archive
from   urllib.request import urlopen
import pathlib
import daft
from   daft import DataType, col
import configparser

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 37, Finished, Available, Finished)

**<mark>Connect to the Catalog</mark>**

In [17]:
config = configparser.ConfigParser()
# this is just a hack, connect to Azure Key valut instead
config.read("/lakehouse/default/Files/KV/variable.ini")
polaris_key = config.get("myvars", "data_engineer")

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 38, Finished, Available, Finished)

In [18]:
def connect_catalog(polaris_key,role):
  catalog = load_catalog(
          'default',
           uri='https://jltvfet-polaris.snowflakecomputing.com/polaris/api/catalog',
           warehouse='dwh',
           scope = f'PRINCIPAL_ROLE:{role}' ,
           credential= polaris_key
        )
  return catalog

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 39, Finished, Available, Finished)

In [19]:
polaris_key = config.get("myvars", "read_scada")
catalog = connect_catalog(polaris_key,"read_scada")

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 40, Finished, Available, Finished)

# Query using Daft

In [20]:
df = daft.read_iceberg(catalog.load_table("demo.scada"))
df.show()

UNITUtf8,VERSIONFloat64,"SETTLEMENTDATETimestamp(Microseconds, None)",RUNNOFloat64,DUIDUtf8,INTERVENTIONFloat64,DISPATCHMODEFloat64,AGCSTATUSFloat64,INITIALMWFloat64,TOTALCLEAREDFloat64,RAMPDOWNRATEFloat64,RAMPUPRATEFloat64,LOWER5MINFloat64,LOWER60SECFloat64,LOWER6SECFloat64,RAISE5MINFloat64,RAISE60SECFloat64,RAISE6SECFloat64,MARGINAL5MINVALUEFloat64,MARGINAL60SECVALUEFloat64,MARGINAL6SECVALUEFloat64,MARGINALVALUEFloat64,VIOLATION5MINDEGREEFloat64,VIOLATION60SECDEGREEFloat64,VIOLATION6SECDEGREEFloat64,VIOLATIONDEGREEFloat64,LOWERREGFloat64,RAISEREGFloat64,AVAILABILITYFloat64,RAISE6SECFLAGSFloat64,RAISE60SECFLAGSFloat64,RAISE5MINFLAGSFloat64,RAISEREGFLAGSFloat64,LOWER6SECFLAGSFloat64,LOWER60SECFLAGSFloat64,LOWER5MINFLAGSFloat64,LOWERREGFLAGSFloat64,RAISEREGAVAILABILITYFloat64,RAISEREGENABLEMENTMAXFloat64,RAISEREGENABLEMENTMINFloat64,LOWERREGAVAILABILITYFloat64,LOWERREGENABLEMENTMAXFloat64,LOWERREGENABLEMENTMINFloat64,RAISE6SECACTUALAVAILABILITYFloat64,RAISE60SECACTUALAVAILABILITYFloat64,RAISE5MINACTUALAVAILABILITYFloat64,RAISEREGACTUALAVAILABILITYFloat64,LOWER6SECACTUALAVAILABILITYFloat64,LOWER60SECACTUALAVAILABILITYFloat64,LOWER5MINACTUALAVAILABILITYFloat64,LOWERREGACTUALAVAILABILITYFloat64,fileUtf8,DATEDate,yearInt32
DUNIT,3,2024-10-28 04:05:00,1,ADPBA1,0,0,1,0.304,0,93.12,93.12,2,2,2,3,3,3,None,None,None,None,None,None,None,None,0,0,6,1,1,1,1,1,1,1,1,7.75999,6,-6,7.75999,6,-6,3,3,3,3,3,3,3,4,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,ADPPV1,0,0,0,0,0,120,120,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,AGLHAL,0,0,0,0,0,720,720,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,AGLSOM,0,0,0,0,0,360,360,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,ANGAST1,0,0,0,0,0,840,840,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,APD01,0,0,0,0,0,0,0,0,0,0,21.61344,1.63534,21,None,None,None,None,None,None,None,None,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,450,450,450,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,ARWF1,0,0,0,102.5,102.46862,240,1200,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,102.46862,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024
DUNIT,3,2024-10-28 04:05:00,1,ASDLBY01,0,0,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,PUBLIC_DAILY_202410280000_20241029040503.CSV,2024-10-28,2024


# Copy Data as Delta Table

In [21]:
from pyiceberg.expressions import EqualTo, NotEqualTo, And
from deltalake.writer import write_deltalake
polaris_key = config.get("myvars", "read_scada")

catalog = connect_catalog(polaris_key,"read_scada")
table = catalog.load_table("demo.scada")
df = table.scan(
    row_filter=And(EqualTo("year", 2024),NotEqualTo("INITIALMW", 0),EqualTo("INTERVENTION", 0)) ,
    selected_fields=("DUID", "INITIALMW", "SETTLEMENTDATE"),
).to_arrow_batch_reader()

write_deltalake("/lakehouse/default/Tables/polaris/scada/", df, mode="append",storage_options= {"allow_unsafe_rename":"true"})

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 42, Finished, Available, Finished)

# Granular Access Control

In [22]:
catalog.list_namespaces()

StatementMeta(, 18c23fc7-8584-418b-a4a2-a57feec02519, 43, Finished, Available, Finished)

ForbiddenError: ForbiddenException: Principal 'read_scada' with activated PrincipalRoles '[read_scada]' and activated ids '[2182423207056970, 2182423207060974]' is not authorized for op LIST_NAMESPACES